# Set cuda

In [1]:
import torch
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"


# Import packages

In [2]:
import detectron2
import os
from detectron2.utils.logger import setup_logger
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.engine import DefaultTrainer
from detectron2.data.datasets import register_coco_instances
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader
from detectron2 import model_zoo
from detectron2.utils.visualizer import ColorMode
from detectron2.modeling import build_model
import cv2
setup_logger()


<Logger detectron2 (DEBUG)>

# Set testing path Register the coco instance

In [3]:
path_to_test_images = "./dataset/test/"
path_to_test_annotations = "./dataset/test_img_ids.json"


# Register the coco instance

In [4]:
register_coco_instances("test", {}, path_to_test_annotations,
                        path_to_test_images)

# Load the trained parameters

In [9]:
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file(
    "COCO-InstanceSegmentation/mask_rcnn_R_101_FPN_3x.yaml"))  # 101
cfg.DATASETS.TEST = ("test", )
cfg.MODEL.WEIGHTS = os.path.join("./output/model_final.pth")
cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 64
cfg.SOLVER.IMS_PER_BATCH = 2
cfg.SOLVER.BASE_LR = 0.00025
cfg.SOLVER.MAX_ITER = 100000
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1
# os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
predictor = DefaultPredictor(cfg)

# Show one prediction about an image

In [10]:
img = cv2.imread("./dataset/test/TCGA-G9-6348-01Z-00-DX1.png")
output = predictor(img)
print(output)

{'instances': Instances(num_instances=100, image_height=1000, image_width=1000, fields=[pred_boxes: Boxes(tensor([[153.5361, 367.8488, 188.6047, 401.1132],
        [199.7998, 672.0380, 236.0706, 705.4072],
        [724.5292,  72.1132, 766.2985,  98.6640],
        [810.4023, 655.4791, 834.3146, 682.6049],
        [263.2572, 410.7906, 287.7430, 446.3961],
        [802.0037, 126.3244, 832.3513, 159.7401],
        [204.5383, 141.4646, 233.5141, 174.0149],
        [610.5238,   3.5143, 637.5432,  35.2490],
        [282.9544, 878.4156, 305.5258, 908.3585],
        [766.5189, 764.5666, 795.4493, 791.2693],
        [858.6918, 348.9867, 900.7940, 383.2085],
        [122.5992, 387.6637, 149.7551, 416.0755],
        [166.9514, 181.4566, 209.4774, 205.7096],
        [319.3407, 463.2062, 353.7127, 497.6846],
        [678.9890, 350.2056, 712.7902, 386.3116],
        [650.4493, 831.2872, 673.9141, 860.5164],
        [ 60.8645, 974.1350,  93.3481, 994.6210],
        [ 14.2592, 914.3790,  44.1129, 937.1

# Function to load the test image json file

In [11]:
def get_test_data(img_dir):
    TEST_SIZE = 6
    json_file_path = os.path.join(img_dir, "test_img_ids.json")
    with open(json_file_path) as f:
        info = json.load(f)
    filenames = []
    imgids = []
    for i in range(TEST_SIZE):
        filenames.append(info[i]['file_name'])
        imgids.append(info[i]['id'])
    for img, file in zip(imgids, filenames):
        yield img, file


# Output the predictions and save as json file

In [12]:
import json
import numpy as np
import pycocotools._mask as _mask
final_list = []

for id, file in get_test_data('./dataset/'):
    im = cv2.imread(f'./dataset/test/{file}')
    # load the image

    outputs = predictor(im)
    # outputs by the predictor
    field_list = outputs['instances'].get_fields()

    bbox_all = field_list['pred_boxes'].tensor
    score_all = field_list['scores']
    seg_all = field_list['pred_masks']
    # save the output data to list
    for ins in range(score_all.shape[0]):
        bbox_ins = bbox_all[ins].tolist()
        score_ins = score_all[ins].item()
        seg_ins = seg_all[ins].cpu()
        seg_ins = np.asfortranarray(seg_ins)
        h, w = seg_ins.shape
        rle_ins = _mask.encode(seg_ins.reshape((h, w, 1), order='F'))[0]
        rle_ins['counts'] = rle_ins['counts'].decode('ascii')
        final_list.append(dict(
            image_id=id,
            bbox=bbox_ins,
            score=score_ins,
            category_id=1,
            segmentation=rle_ins
        ))

print(final_list)

with open('./output/answer_iter.json', 'w') as f:
    json.dump(final_list, f)


[{'image_id': 1, 'bbox': [763.7533569335938, 646.3599853515625, 806.5863037109375, 693.4791259765625], 'score': 1.0, 'category_id': 1, 'segmentation': {'size': [1000, 1000], 'counts': 'kgZg0:kn07J4L4L3N3M2N2N1O2N2O0O1O100O100O1O10000O0100O10000O1O100O2N1O2O0O2N2N2M3N2M3M3M4K7Ghjk5'}}, {'image_id': 1, 'bbox': [724.7449340820312, 747.4542846679688, 769.1475830078125, 794.9697265625], 'score': 1.0, 'category_id': 1, 'segmentation': {'size': [1000, 1000], 'counts': '_gUf0`0dn05M4M1N3N2N3M101N1O2O0O100O2O0O1O10000O10O100000000O10000O2O0O2N2N2O1M3N3M3L4M6G_iR7'}}, {'image_id': 1, 'bbox': [258.77545166015625, 957.5438232421875, 287.44378662109375, 988.2833862304688], 'score': 0.9999998807907104, 'category_id': 1, 'segmentation': {'size': [1000, 1000], 'counts': 'Xlm7:mn01O1O1O100O001OO10000000001O1O0O2O1O1O1N2O1O00001O001OY[ge0'}}, {'image_id': 1, 'bbox': [332.3868408203125, 952.5560302734375, 351.0028076171875, 987.88525390625], 'score': 0.9999998807907104, 'category_id': 1, 'segmentation': 